In [0]:
dbutils.secrets.help()

Provides utilities for leveraging secrets within notebooks.
Databricks documentation for more info.
 get(scope: String, key: String): String -> Gets the string representation of a secret value with scope and key getBytes(scope: String, key: String): byte[] -> Gets the bytes representation of a secret value with scope and key list(scope: String): Seq -> Lists secret metadata for secrets within a scope listScopes: Seq -> Lists secret scopes

In [0]:
dbutils.secrets.listScopes()

[SecretScope(name='usaccidents-scope')]

In [0]:
dbutils.secrets.list(scope = 'usaccidents-scope')

[SecretMetadata(key='db-app-clientid'),
 SecretMetadata(key='db-app-clientsecret'),
 SecretMetadata(key='db-app-tenentid')]

In [0]:
dbutils.secrets.get(scope = 'usaccidents-scope', key = 'db-app-clientid')

'[REDACTED]'

In [0]:
storage_account_name = "usaccidentdl"
client_id = dbutils.secrets.get(scope="usaccidents-scope", key="db-app-clientid")
tenent_id = dbutils.secrets.get(scope="usaccidents-scope", key="db-app-tenentid")
client_secret = dbutils.secrets.get(scope="usaccidents-scope", key="db-app-clientsecret")

In [0]:
configs = {"fs.azure.account.auth.type":"OAuth",
           "fs.azure.account.oauth.provider.type":"org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": f"{client_id}",
           "fs.azure.account.oauth2.client.secret":f"{client_secret}",
           "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenent_id}/oauth2/token"
    
}

In [0]:
def mount_adls(container_name):
    dbutils.fs.mount(
        source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
        mount_point = f"/mnt/{storage_account_name}/{container_name}",
        extra_configs = configs)

In [0]:
mount_adls("raw")

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
File <command-1358272751373578>, line 1
----> 1 mount_adls("/mnt")

File <command-1358272751373577>, line 2, in mount_adls(container_name)
      1 def mount_adls(container_name):
----> 2     dbutils.fs.mount(
      3         source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
      4         mount_point = f"/mnt/{storage_account_name}/{container_name}",
      5         extra_configs = configs)

File /databricks/python_shell/dbruntime/dbutils.py:346, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    344 def f_with_exception_handling(*args, **kwargs):
    345     try:
--> 346         return f(*args, **kwargs)
    347     except Py4JJavaError as e:
    349         class ExecutionError(Exception):

File /databricks/python_shell/dbruntime/dbutils.py:451, in 

In [0]:
mount_adls('processed')

In [0]:
dbutils.fs.ls("dbfs:/mnt/usaccidentdl")

[FileInfo(path='dbfs:/mnt/usaccidentdl/processed/', name='processed/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/usaccidentdl/raw/', name='raw/', size=0, modificationTime=0)]